In [23]:
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
X_train=pd.read_csv('./data/train.csv',index_col='Webpage_id')
y_train=X_train['Tag']
X_train.drop(columns=['Tag'], inplace=True)
X_test=pd.read_csv('./data/test.xls',index_col='Webpage_id')

In [3]:
urlp=X_train["Url"]
urlp = urlp.replace('((www\.)|(http?://)|(.com/))',' ',regex=True)
special_char_list = ['\.', '\;', '\?', '\}', '\)', '\{', '\(','/','\-']
for special_char in special_char_list:
    urlp = urlp.replace(special_char, ' ',regex=True)
X_train['Url']=urlp


urlpt=X_test["Url"]
urlpt = urlpt.replace('((www\.)|(http?://)|(.com/)|(https?://))',' ',regex=True)
special_char_list = ['\.', '\;', '\?', '\}', '\)', '\{', '\(','/','\-']
for special_char in special_char_list:
    urlpt = urlpt.replace(special_char, ' ',regex=True)
X_test['Url']=urlpt

In [4]:
y_train=list(y_train)
X_train=list(X_train.iloc[:,-1])
X_test1=list(X_test.iloc[:,-1])

In [5]:
#html_data = pd.read_csv('C:/Users/Hardika/Downloads/InnoplexH/html_data.csv')
#html_data.head()
#from bs4 import BeautifulSoup
#soup = BeautifulSoup(html_data['Html'][0], "html5lib")
#print(soup.find(name='title'))
#print(soup.find(name='title'))
#print(soup.findAll('articleBody'))

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer()
# use TreeankWordTokenizer
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
vect.set_params(tokenizer=tokenizer.tokenize)

# remove English stop words
vect.set_params(stop_words='english')

# include 1-grams and 2-grams consider both grams
vect.set_params(ngram_range=(1, 2))

# ignore terms that appear in more than 50% of the documents
vect.set_params(max_df=0.5)

# learn the 'vocabulary' of the training data
vect.fit(X_train)

# examine the fitted vocabulary
#vect.get_feature_names()[:50]

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x0000020AD4C37438>>,
        vocabulary=None)

In [6]:
train_vectors = vect.transform(X_train)
test_vectors1 = vect.transform(X_test1)

In [7]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
nb = MultinomialNB()

In [8]:
nb.fit(train_vectors, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
from sklearn.metrics.classification import classification_report, accuracy_score
from sklearn.cross_validation import cross_val_predict

C:\Users\Hardika\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
preds = cross_val_predict(nb,train_vectors,y_train,cv=10)
print(classification_report(y_train, preds))
print("ACCURACY::", accuracy_score(preds, y_train))

                precision    recall  f1-score   support

clinicalTrials       0.90      0.99      0.94      2839
   conferences       0.82      0.85      0.83      4666
         forum       0.85      0.93      0.89      4503
    guidelines       0.80      0.77      0.78      1329
          news       0.64      0.70      0.67      7992
        others       0.78      0.64      0.70     17417
       profile       0.76      0.76      0.76      5196
   publication       0.81      0.95      0.87      7705
        thesis       0.91      1.00      0.95      1800

   avg / total       0.78      0.78      0.78     53447

ACCURACY:: 0.7821206054596141


In [11]:
y_pred1=nb.predict(test_vectors1)

In [27]:
S11 = X_test.index.values
S22 = pd.DataFrame(y_pred1)

FR= pd.DataFrame()
FR['Webpage_id']=S11
FR['Tag']=S22

FR.to_csv('./Submission/Summary_SubmissionF1.csv')


In [24]:
svcl = SGDClassifier(loss='hinge',penalty='l2',alpha=1e-3,n_iter=5,random_state=9)
svcl.fit(train_vectors, y_train)
predsvcl = cross_val_predict(svcl,train_vectors,y_train,cv=10)
print(classification_report(y_train, predsvcl))
print("ACCURACY::", accuracy_score(predsvcl, y_train))

                precision    recall  f1-score   support

clinicalTrials       0.99      0.99      0.99      2839
   conferences       0.91      0.84      0.88      4666
         forum       0.98      0.74      0.84      4503
    guidelines       0.87      0.82      0.84      1329
          news       0.80      0.67      0.73      7992
        others       0.71      0.89      0.79     17417
       profile       0.88      0.63      0.74      5196
   publication       0.91      0.90      0.90      7705
        thesis       0.99      1.00      0.99      1800

   avg / total       0.84      0.82      0.82     53447

ACCURACY:: 0.8243867756843228


In [25]:
y_pred2=svcl.predict(test_vectors1)
svcI = X_test.index.values
svcT = pd.DataFrame(y_pred2)

FRSvc= pd.DataFrame()
FRSvc['Webpage_id']=svcI
FRSvc['Tag']=svcT

FRSvc.to_csv('./Submission/Summary_SubmissionFSgd.csv')

In [16]:
OvR = OneVsRestClassifier(LogisticRegression(random_state=9))
OvR.fit(train_vectors, y_train)
predsOvR = cross_val_predict(OvR,train_vectors,y_train,cv=10)
print(classification_report(y_train, predsOvR))
print("ACCURACY::", accuracy_score(predsOvR, y_train))

                precision    recall  f1-score   support

clinicalTrials       0.99      0.99      0.99      2839
   conferences       0.90      0.89      0.90      4666
         forum       0.98      0.77      0.86      4503
    guidelines       0.84      0.85      0.84      1329
          news       0.78      0.69      0.73      7992
        others       0.74      0.86      0.80     17417
       profile       0.87      0.68      0.77      5196
   publication       0.90      0.92      0.91      7705
        thesis       1.00      1.00      1.00      1800

   avg / total       0.84      0.83      0.83     53447

ACCURACY:: 0.8339289389488652


In [26]:
y_pred3=OvR.predict(test_vectors1)
OvrI = X_test.index.values
OvrT = pd.DataFrame(y_pred3)

FROvr= pd.DataFrame()
FROvr['Webpage_id']=OvrI
FROvr['Tag']=OvrT

FROvr.to_csv('./Submission/Summary_SubmissionFOvr.csv')

In [ ]:
Finalized OneVsRestClassifier Model